<a href="https://colab.research.google.com/github/shstreuber/AI/blob/main/Week12_DigitsGAN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#**Generative Adversarial Network**

Taken from Datacamp tutorial: https://www.datacamp.com/tutorial/generative-adversarial-networks

Explanations by Professor Sonja Streuber

##**0. Import Libraries and Set Up Environment**

The code below imports libraries needed to build and train a neural network using Keras.

* `os, numpy, matplotlib.pyplot`, and `tqdm` help with file handling, data manipulation, visualization, and displaying progress bars.

* `Input, Dense, Dropout,` and `LeakyReLU` are used to define layers in the neural network.

* `mnist` is a dataset of handwritten digits used for training and testing the model.

* `Adam` is an optimizer used to improve model performance during training.

In summary, this code prepares for building a neural network to classify handwritten digits from the MNIST dataset.

In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm


from keras.layers import Input
from keras.models import Model, Sequential
from keras.layers.core import Dense, Dropout
#from keras.layers.advanced_activations import LeakyReLU
from keras.layers import LeakyReLU
from keras.datasets import mnist
from keras.optimizers import Adam
from keras import initializers

##**0.1 Set Up Reproducibility and Load Dataset**

In [ ]:
# To make sure that we can reproduce the experiment and get the same results
# Use last 2 digits of your student ID for the seed value.
np.random.seed(10)

# The dimension of our random noise vector.
random_dim = 100

In [ ]:
def load_minst_data():
    # load the data
    (x_train, y_train), (x_test, y_test) = mnist.load_data()
    # normalize our inputs to be in the range[-1, 1]
    x_train = (x_train.astype(np.float32) - 127.5)/127.5
    # convert x_train with a shape of (60000, 28, 28) to (60000, 784) so we have
    # 784 columns per row
    x_train = x_train.reshape(60000, 784)
    return (x_train, y_train, x_test, y_test)

In the block above, the `load_mnist_data` function loads the MNIST dataset, which consists of handwritten digits, using Keras' `mnist.load_data()`. It then normalizes the training images (`x_train`) to have values in the range of [-1, 1] by subtracting 127.5 and dividing by 127.5. The function reshapes the input data from a 3D array of shape (60000, 28, 28) into a 2D array of shape (60000, 784), where each 28x28 image is flattened into a 784-dimensional vector. Finally, the function returns the normalized and reshaped training data, along with the corresponding labels (`y_train`), as well as the test data and labels (`x_test`, `y_test`).

##**1. Build Generative Adversarial Network**
Now, we are going to define the functions to create the optimizer, generator, and discriminator models used in a Generative Adversarial Network (GAN):

###**1.1 Define Optimizer**
The `get_optimizer` function defines and returns an Adam optimizer for training.

In [ ]:
def get_optimizer():
    return Adam(lr=0.0002, beta_1=0.5)

* `Adam`: The Adam optimizer is an adaptive learning rate optimizer that adjusts the learning rate for each parameter, which makes it efficient and effective for large datasets and models.

* `lr=0.0002`: The learning rate is set to 0.0002, which controls how much the model weights should be adjusted during training.

* `beta_1=0.5`: This parameter is the exponential decay rate for the first moment estimate (the moving average of the gradient). It is set to 0.5 to make the optimizer more responsive to updates during training.

The optimizer is returned to be used when compiling models.



###**1.2 Build the Generator**

Next, we will build the `get_generator` function.

In [ ]:
def get_generator(optimizer):
    generator = Sequential()
    generator.add(Dense(256, input_dim=random_dim, kernel_initializer=initializers.RandomNormal(stddev=0.02)))
    generator.add(LeakyReLU(0.2))

    generator.add(Dense(512))
    generator.add(LeakyReLU(0.2))

    generator.add(Dense(1024))
    generator.add(LeakyReLU(0.2))

    generator.add(Dense(784, activation='tanh'))
    generator.compile(loss='binary_crossentropy', optimizer=optimizer)
    return generator

This function builds and returns a generator model, which is part of a Generative Adversarial Network (GAN).

**Explanation**:

* `Sequential()`: This indicates that the model is built layer by layer in a linear stack.

* `Dense(256, input_dim=random_dim)`: The first layer is a dense (fully connected) layer with 256 units. The input dimension is random_dim, which typically represents the size of the random noise vector used to generate data.

* `kernel_initializer=initializers.RandomNormal(stddev=0.02)`: This initializes the weights of the layer with a normal distribution and a standard deviation of 0.02, which helps in breaking symmetry and improving learning.

* `LeakyReLU(0.2)`: This is an activation function that introduces non-linearity. It uses a small slope (0.2) for negative values, avoiding the "dying ReLU" problem.

The model continues adding dense layers of sizes 512 and 1024, each followed by LeakyReLU activation.

* `Dense(784, activation='tanh')`: The final output layer generates 784 outputs, corresponding to a 28x28 image (784 pixels). The tanh activation ensures the outputs are scaled between -1 and 1.

* `generator.compile(loss='binary_crossentropy', optimizer=optimizer)`: The generator is compiled with a binary cross-entropy loss function (used for binary classification) and the Adam optimizer passed as an argument.

Finally, the generator model is returned.

###**1.3 Build the Discriminator**

Lastly, we need the `get_discriminator` function.

In [ ]:
def get_discriminator(optimizer):
    discriminator = Sequential()
    discriminator.add(Dense(1024, input_dim=784, kernel_initializer=initializers.RandomNormal(stddev=0.02)))
    discriminator.add(LeakyReLU(0.2))
    discriminator.add(Dropout(0.3))

    discriminator.add(Dense(512))
    discriminator.add(LeakyReLU(0.2))
    discriminator.add(Dropout(0.3))

    discriminator.add(Dense(256))
    discriminator.add(LeakyReLU(0.2))
    discriminator.add(Dropout(0.3))

    discriminator.add(Dense(1, activation='sigmoid'))
    discriminator.compile(loss='binary_crossentropy', optimizer=optimizer)
    return discriminator

The `get_discriminator` function builds and returns a discriminator model, another key component of a Generative Adversarial Network (GAN).

**Explanation**:

* `Sequential()`: The discriminator model is built layer by layer.

* `Dense(1024, input_dim=784)`: The first layer is a dense layer with 1024 units and an input dimension of 784 (each input corresponds to a flattened 28x28 image).

* `LeakyReLU(0.2)`: As in the generator, this activation function is used to add non-linearity with a small slope (0.2) for negative values.

* `Dropout(0.3)`: Dropout is applied to each layer to prevent overfitting. It randomly drops 30% of the connections during training.

The model continues with two more dense layers of sizes 512 and 256, each followed by LeakyReLU activation and Dropout.

* `Dense(1, activation='sigmoid')`: The output layer has a single unit with a sigmoid activation function. This layer outputs a probability (between 0 and 1) indicating whether the input image is real (1) or fake (0).

* `discriminator.compile(loss='binary_crossentropy', optimizer=optimizer)`: The discriminator is compiled with a binary cross-entropy loss function (for binary classification) and the Adam optimizer passed as an argument.

Finally, the discriminator model is returned.

###**1.4 Assemble and Compile the GAN**

The following `get_gan_network` function defines and returns a Generative Adversarial Network (GAN) model that combines a generator and a discriminator to create an adversarial system.

In [ ]:
def get_gan_network(discriminator, random_dim, generator, optimizer):
    # We initially set trainable to False since we only want to train either the
    # generator or discriminator at a time
    discriminator.trainable = False
    # gan input (noise) will be 100-dimensional vectors
    gan_input = Input(shape=(random_dim,))
    # the output of the generator (an image)
    x = generator(gan_input)
    # get the output of the discriminator (probability if the image is real or not)
    gan_output = discriminator(x)
    gan = Model(inputs=gan_input, outputs=gan_output)
    gan.compile(loss='binary_crossentropy', optimizer=optimizer)
    return gan

The `get_gan_network` function above creates and compiles a Generative Adversarial Network (GAN). It takes the discriminator, random noise dimension, generator, and optimizer as inputs. The function first sets the discriminator's weights to be non-trainable, ensuring that only the generator is updated when training the GAN. It then defines the input layer for the GAN, which accepts random noise vectors. This noise is passed through the generator to produce synthetic images. The generated images are then passed to the discriminator, which evaluates whether they are real or fake. The function creates a model that connects the input (random noise) to the output (discriminator's decision), compiles the model with a binary cross-entropy loss function and the provided optimizer, and finally returns the compiled GAN model, ready for training.

##**2. Plotting the Generated Images**

The `plot_generated_images` function below generates and saves a grid of synthetic MNIST images created by the generator. It takes several inputs: the current epoch number, the generator model, the number of images to generate (`examples`), the grid dimensions (`dim`), and the figure size (`figsize`). Inside the function, random noise vectors are generated and passed through the generator to create fake MNIST images. These images are reshaped to a 28x28 format and arranged in a grid based on the specified dimensions. The function then plots each image in the grid, disables the axis labels for a cleaner look, and saves the resulting image grid to a file with the epoch number in the filename. This allows tracking the progression of the generator's ability to produce realistic images as training progresses.

In [ ]:
# Create a wall of generated MNIST images
def plot_generated_images(epoch, generator, examples=20, dim=(10, 10), figsize=(10, 10)):
    noise = np.random.normal(0, 1, size=[examples, random_dim])
    generated_images = generator.predict(noise)
    generated_images = generated_images.reshape(examples, 28, 28)

    plt.figure(figsize=figsize)
    for i in range(generated_images.shape[0]):
        plt.subplot(dim[0], dim[1], i+1)
        plt.imshow(generated_images[i], interpolation='nearest', cmap='gray_r')
        plt.axis('off')
    plt.tight_layout()
    plt.savefig('gan_generated_image_epoch_%d.png' % epoch)

##**3. Training and Evaluating the GAN**

In the code below, the `train` function is responsible for training the GAN (Generative Adversarial Network) on the MNIST dataset. Here's a breakdown of what the code does:

1. **Loading Data:** It first loads the MNIST dataset using `load_minst_data()`, which returns the training and testing data.

2. **Batch Splitting:** The training data is divided into batches of size `batch_size` (default: 128). The `batch_count` is calculated based on the number of training samples divided by the batch size.

3. **Model Initialization:** The optimizer (`adam`) is created using the `get_optimizer()` function. Then, the generator, discriminator, and GAN models are built using `get_generator()`, `get_discriminator()`, and `get_gan_network()` respectively.

4. **Training Loop:** The training proceeds for the specified number of epochs:
   - For each epoch, the function loops over batches of the training data.
   - Random noise is generated to serve as input to the generator, and random real MNIST images are selected from the training data.
   - **Discriminator Training:** The discriminator is trained on both real and generated images. The real images are labeled with values close to 1 (real), and the generated images are labeled as 0 (fake) with some label smoothing applied (real images are labeled 0.9 instead of 1).
   - **Generator Training:** After training the discriminator, the generator is trained to fool the discriminator. The noise input is passed through the generator, and the output is labeled as real (`y_gen = np.ones(batch_size)`), but the discriminator is kept frozen (`trainable = False`) during this training.

5. **Image Generation (Optional):** The commented-out section is for plotting generated images periodically during training. It would save images generated at the first epoch or every 20th epoch to monitor progress. In the final step, after all epochs, the `plot_generated_images()` function is called to save a final grid of generated images.

In summary, the function implements the core GAN training process where the discriminator and generator are alternately trained. The discriminator learns to distinguish between real and fake images, while the generator tries to produce more realistic fake images over time. The result is the training of both models to improve the generator's ability to create convincing MNIST-like images.

In [ ]:
def train(epochs=1, batch_size=128):
    # Get the training and testing data
    x_train, y_train, x_test, y_test = load_minst_data()
    # Split the training data into batches of size 128
    batch_count = x_train.shape[0] // batch_size

    # Build our GAN netowrk
    adam = get_optimizer()
    generator = get_generator(adam)
    discriminator = get_discriminator(adam)
    gan = get_gan_network(discriminator, random_dim, generator, adam)

    for e in range(1, epochs+1):
        print('-'*15, 'Epoch %d' % e, '-'*15)
        for z in tqdm(range(batch_count)):
            # Get a random set of input noise and images
            noise = np.random.normal(0, 1, size=[batch_size, random_dim])
            image_batch = x_train[np.random.randint(0, x_train.shape[0], size=batch_size)]

            # Generate fake MNIST images
            generated_images = generator.predict(noise)
            X = np.concatenate([image_batch, generated_images])

            # Labels for generated and real data
            y_dis = np.zeros(2*batch_size)
            # One-sided label smoothing
            y_dis[:batch_size] = 0.9

            # Train discriminator
            discriminator.trainable = True
            discriminator.train_on_batch(X, y_dis)

            # Train generator
            noise = np.random.normal(0, 1, size=[batch_size, random_dim])
            y_gen = np.ones(batch_size)
            discriminator.trainable = False
            gan.train_on_batch(noise, y_gen)
        #
        #if e == 1 or e % 20 == 0:
        #    plot_generated_images(e, generator)
    #Plot using the final epoch
    plot_generated_images(epochs, generator)

##**4. Adjusting the Training**

In [ ]:
train(2,128)

The code `train(2, 128)` calls the `train` function with two arguments:

1. **`epochs=2`**: This specifies that the training process should run for 2 epochs. An epoch means one complete pass through the entire training dataset. So, the model will go through the full MNIST dataset twice.

2. **`batch_size=128`**: This specifies that the training will be done in batches of 128 samples at a time. During each batch, 128 random noise vectors and 128 real MNIST images are used to train the discriminator and the generator.

In essence, `train(2, 128)` starts the GAN training process, running for 2 epochs with a batch size of 128. Each epoch consists of multiple iterations where the discriminator and generator are trained alternately. The model will learn to generate fake MNIST-like images, and the training process will print the progress of the epochs and batches. After the training, it will also generate and save images of the results at the end of the 2 epochs.